# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f79543b3f10>
├── 'a' --> tensor([[-1.4767, -1.4637,  0.3815],
│                   [-0.5734,  0.9261,  0.1562]])
└── 'x' --> <FastTreeValue 0x7f79543b3d90>
    └── 'c' --> tensor([[ 0.2640, -0.4432,  0.0406, -0.3534],
                        [ 0.8988, -0.3702, -0.7845, -0.1178],
                        [ 0.7269,  0.2695,  0.7317, -2.5379]])

In [4]:
t.a

tensor([[-1.4767, -1.4637,  0.3815],
        [-0.5734,  0.9261,  0.1562]])

In [5]:
%timeit t.a

64.4 ns ± 0.11 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f79543b3f10>
├── 'a' --> tensor([[-0.2038,  0.7590,  0.1471],
│                   [ 0.3643, -0.4599,  1.1088]])
└── 'x' --> <FastTreeValue 0x7f79543b3d90>
    └── 'c' --> tensor([[ 0.2640, -0.4432,  0.0406, -0.3534],
                        [ 0.8988, -0.3702, -0.7845, -0.1178],
                        [ 0.7269,  0.2695,  0.7317, -2.5379]])

In [7]:
%timeit t.a = new_value

66.1 ns ± 0.0633 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.4767, -1.4637,  0.3815],
               [-0.5734,  0.9261,  0.1562]]),
    x: Batch(
           c: tensor([[ 0.2640, -0.4432,  0.0406, -0.3534],
                      [ 0.8988, -0.3702, -0.7845, -0.1178],
                      [ 0.7269,  0.2695,  0.7317, -2.5379]]),
       ),
)

In [10]:
b.a

tensor([[-1.4767, -1.4637,  0.3815],
        [-0.5734,  0.9261,  0.1562]])

In [11]:
%timeit b.a

58 ns ± 0.176 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7027, -1.1101, -0.7077],
               [-0.5650, -0.3460,  0.3861]]),
    x: Batch(
           c: tensor([[ 0.2640, -0.4432,  0.0406, -0.3534],
                      [ 0.8988, -0.3702, -0.7845, -0.1178],
                      [ 0.7269,  0.2695,  0.7317, -2.5379]]),
       ),
)

In [13]:
%timeit b.a = new_value

489 ns ± 0.143 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

824 ns ± 3.37 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 18.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 4.38 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 948 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7954397c10>
├── 'a' --> tensor([[[-1.4767, -1.4637,  0.3815],
│                    [-0.5734,  0.9261,  0.1562]],
│           
│                   [[-1.4767, -1.4637,  0.3815],
│                    [-0.5734,  0.9261,  0.1562]],
│           
│                   [[-1.4767, -1.4637,  0.3815],
│                    [-0.5734,  0.9261,  0.1562]],
│           
│                   [[-1.4767, -1.4637,  0.3815],
│                    [-0.5734,  0.9261,  0.1562]],
│           
│                   [[-1.4767, -1.4637,  0.3815],
│                    [-0.5734,  0.9261,  0.1562]],
│           
│                   [[-1.4767, -1.4637,  0.3815],
│                    [-0.5734,  0.9261,  0.1562]],
│           
│                   [[-1.4767, -1.4637,  0.3815],
│                    [-0.5734,  0.9261,  0.1562]],
│           
│                   [[-1.4767, -1.4637,  0.3815],
│                    [-0.5734,  0.9261,  0.1562]]])
└── 'x' --> <FastTreeValue 0x7f78876b2520>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 57.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f795743ae50>
├── 'a' --> tensor([[-1.4767, -1.4637,  0.3815],
│                   [-0.5734,  0.9261,  0.1562],
│                   [-1.4767, -1.4637,  0.3815],
│                   [-0.5734,  0.9261,  0.1562],
│                   [-1.4767, -1.4637,  0.3815],
│                   [-0.5734,  0.9261,  0.1562],
│                   [-1.4767, -1.4637,  0.3815],
│                   [-0.5734,  0.9261,  0.1562],
│                   [-1.4767, -1.4637,  0.3815],
│                   [-0.5734,  0.9261,  0.1562],
│                   [-1.4767, -1.4637,  0.3815],
│                   [-0.5734,  0.9261,  0.1562],
│                   [-1.4767, -1.4637,  0.3815],
│                   [-0.5734,  0.9261,  0.1562],
│                   [-1.4767, -1.4637,  0.3815],
│                   [-0.5734,  0.9261,  0.1562]])
└── 'x' --> <FastTreeValue 0x7f7957441be0>
    └── 'c' --> tensor([[ 0.2640, -0.4432,  0.0406, -0.3534],
                        [ 0.8988, -0.3702, -0.7845, -0.1178],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 54.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

65.8 µs ± 101 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2640, -0.4432,  0.0406, -0.3534],
                       [ 0.8988, -0.3702, -0.7845, -0.1178],
                       [ 0.7269,  0.2695,  0.7317, -2.5379]],
              
                      [[ 0.2640, -0.4432,  0.0406, -0.3534],
                       [ 0.8988, -0.3702, -0.7845, -0.1178],
                       [ 0.7269,  0.2695,  0.7317, -2.5379]],
              
                      [[ 0.2640, -0.4432,  0.0406, -0.3534],
                       [ 0.8988, -0.3702, -0.7845, -0.1178],
                       [ 0.7269,  0.2695,  0.7317, -2.5379]],
              
                      [[ 0.2640, -0.4432,  0.0406, -0.3534],
                       [ 0.8988, -0.3702, -0.7845, -0.1178],
                       [ 0.7269,  0.2695,  0.7317, -2.5379]],
              
                      [[ 0.2640, -0.4432,  0.0406, -0.3534],
                       [ 0.8988, -0.3702, -0.7845, -0.1178],
                       [ 0.7269,  0.2695,  0.7317, -2.5379]],

In [26]:
%timeit Batch.stack(batches)

78.5 µs ± 125 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2640, -0.4432,  0.0406, -0.3534],
                      [ 0.8988, -0.3702, -0.7845, -0.1178],
                      [ 0.7269,  0.2695,  0.7317, -2.5379],
                      [ 0.2640, -0.4432,  0.0406, -0.3534],
                      [ 0.8988, -0.3702, -0.7845, -0.1178],
                      [ 0.7269,  0.2695,  0.7317, -2.5379],
                      [ 0.2640, -0.4432,  0.0406, -0.3534],
                      [ 0.8988, -0.3702, -0.7845, -0.1178],
                      [ 0.7269,  0.2695,  0.7317, -2.5379],
                      [ 0.2640, -0.4432,  0.0406, -0.3534],
                      [ 0.8988, -0.3702, -0.7845, -0.1178],
                      [ 0.7269,  0.2695,  0.7317, -2.5379],
                      [ 0.2640, -0.4432,  0.0406, -0.3534],
                      [ 0.8988, -0.3702, -0.7845, -0.1178],
                      [ 0.7269,  0.2695,  0.7317, -2.5379],
                      [ 0.2640, -0.4432,  0.0406, -0.3534],
                   

In [28]:
%timeit Batch.cat(batches)

146 µs ± 750 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

339 µs ± 2.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
